# Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
start = time.time()

---

# Read Files

In [3]:
# List of Excel Files
sales_lst = sorted([doc_ for doc_ in os.listdir("../data") if doc_.startswith("Demand Sensing Sales History") or doc_.startswith("Demand_Sensing_Sales_")])
saha_aktivite_lst = sorted([doc_ for doc_ in os.listdir("../data") if doc_.startswith("Demand_Sensing_Saha_Aktivit") or doc_.startswith("Demand Sensing Saha Aktivit")])
pasifik_aktivite_lst = sorted([doc_ for doc_ in os.listdir("../data") if doc_.startswith("Pasifik Aktivite Datası")])
fiyat_lst = sorted([doc_ for doc_ in os.listdir("../data") if "Fiyat List" in doc_])
portfoy_lst = sorted([doc_ for doc_ in os.listdir("../data") if doc_.startswith("Portföy")])
eslenik_kod_lst = sorted([doc_ for doc_ in os.listdir("../data") if doc_.startswith("Ürün Eşlenik kodlar")])

---
# Read Data
---

In [4]:
# Read Sales Data
pasifik_df_all = []
btt_df_all = []
horizon_saha_df_all = []
chng_cols_beginning = {'Year': 'Yıl', 'Quarter': 'Çeyrek', 'Month': 'Ay', 
 'Company Code': 'Şirket Kodu', 'Main Category Name': 'Ana Kategori Adı', 
 'Category Name': 'Kategori Adı', 'Brand Name': 'Marka Adı', 'Product Code': 'Ürün Kodu', 
 'Product Name': 'Ürün Adı', "Ürün Adı (Mobis)": 'Ürün Adı'}
for docs_ in sales_lst:
    pasifik_df_all.append(pd.read_excel("../data/"+docs_, skiprows=1, sheet_name="Pasifik SAP Satış", usecols="B:P").rename(columns=chng_cols_beginning))
    btt_df_all.append(pd.read_excel("../data/"+docs_, skiprows=1, sheet_name="BTT SAP Satış", usecols="B:N").rename(columns=chng_cols_beginning))
    horizon_saha_df_all.append(pd.read_excel("../data/"+docs_, skiprows=1, sheet_name="Horizon Saha Satış", usecols="B:N").rename(columns=chng_cols_beginning))
pasifik_df_all = pd.concat(pasifik_df_all)
btt_df_all = pd.concat(btt_df_all)
horizon_saha_df_all = pd.concat(horizon_saha_df_all)


chng_cols = {"Saha Müşteri Grup": "Grup adı", "Ürün Adı (Mobis)": "Ürün Adı"}

pasifik_df_all.rename(columns=chng_cols, inplace=True)
btt_df_all.rename(columns=chng_cols, inplace=True)
horizon_saha_df_all.rename(columns=chng_cols, inplace=True)

btt_df_all["Grup adı"], btt_df_all["Grup Kodu."] = "BTT", 0
horizon_saha_df_all["Şirket Kodu"], horizon_saha_df_all["Grup Kodu."] = 0, 0

btt_df_all = btt_df_all[pasifik_df_all.columns.to_list()]
horizon_saha_df_all = horizon_saha_df_all[pasifik_df_all.columns.to_list()]

enflasyon_df = pd.read_excel("../data/enflasyon.xlsx")

yarisma_df = pd.read_excel("../data/2018-2021 Yarışmaları_v2.xlsx", sheet_name="yarisma_historik_data")

FileNotFoundError: [Errno 2] No such file or directory: '2018-2021 Yarışmaları_v2.xlsx'

In [6]:
# Read Saha Aktiviteleri
saha_aktivite_detay = []
saha_aktivite_cat = []

for docs_ in saha_aktivite_lst:
    saha_aktivite_detay.append(pd.read_excel("../data/"+docs_, skiprows=1, sheet_name="Ürün Detay", usecols="B:M"))
    saha_aktivite_cat.append(pd.read_excel("../data/"+docs_, skiprows=1, sheet_name="Kategori", usecols="B:I"))
saha_aktivite_detay = pd.concat(saha_aktivite_detay)
saha_aktivite_cat = pd.concat(saha_aktivite_cat)

In [7]:
# Read Pasifik Aktiviteleri
pasifik_aktivite_df = pd.read_excel("../data/"+pasifik_aktivite_lst[0])

In [8]:
# Read Fiyat Listesi
fiyat_lst_pasifik = pd.read_excel("../data/"+fiyat_lst[0])
fiyat_lst_horizon = pd.read_excel("../data/"+fiyat_lst[1])

In [9]:
# Read Portföy
pasifik_portfoy_df = pd.read_excel("../data/"+portfoy_lst[0], sheet_name="Pasifik Portföy", skiprows=3, usecols="D:H")
btt_portfoy_df = pd.read_excel("../data/"+portfoy_lst[0], sheet_name="BTT Portföy", skiprows=2, usecols="D:H")
horizon_portfoy_df = pd.read_excel("../data/"+portfoy_lst[0], sheet_name="Horizon Portföy", skiprows=2, usecols="E:I")

In [10]:
# Read Eşlenik Kodları
eslenik_kod_df = pd.read_excel("../data/"+eslenik_kod_lst[0])

In [11]:
# Read Calender
calender_df = pd.read_excel("../data/Calender_Monthly.xlsx")

In [12]:
# Read Plasiyer Yarışma
plasiyer_yarisma = pd.read_excel("../data/2018-2021 Yarışmaları.xlsx")

In [13]:
eslenik_kod_df["En Güncel Kod"] = eslenik_kod_df["En Güncel Kod"].apply(lambda x: int(x) if x not in ['delist ', "delist"] else x.replace(" ", ""))

---
# Pasifik 2016 aktivite verileri olmadığı için 2016 Sales dataları çıkartıldı.

In [14]:
pasifik_df_all = pasifik_df_all[pasifik_df_all["Yıl"] != 2016].reset_index(drop=True)

---

---

# Sales Datası İçin Ürün Kod Eşleme

---

---

## Join

In [15]:
pasifik_df_all = pd.merge(pasifik_df_all, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")
btt_df_all = pd.merge(btt_df_all, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")
horizon_saha_df_all = pd.merge(horizon_saha_df_all, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")

In [16]:
pasifik_eslenik_kod_olmayan = pasifik_df_all[pd.isnull(pasifik_df_all["En Güncel Kod"])]
pasifik_eslenik_kod_olmayan_unique = pd.DataFrame(pasifik_df_all[pd.isnull(pasifik_df_all["En Güncel Kod"])]["Ürün Kodu"].unique(), columns=["Ürün Kodu"])

writer_pasifik = pd.ExcelWriter("../reports/pasifik_eslenik_kodu_olmayan.xlsx", engine = 'xlsxwriter')

pasifik_eslenik_kod_olmayan_unique.to_excel(writer_pasifik, sheet_name = 'eslenik_kod_olmayan_sku',index=False)
pasifik_eslenik_kod_olmayan.to_excel(writer_pasifik, sheet_name = 'eslenik_kod_olmayan_data',index=False)

writer_pasifik.save()
writer_pasifik.close()

In [17]:
btt_eslenik_kod_olmayan = btt_df_all[pd.isnull(btt_df_all["En Güncel Kod"])]
btt_eslenik_kod_olmayan_unique = pd.DataFrame(btt_df_all[pd.isnull(btt_df_all["En Güncel Kod"])]["Ürün Kodu"].unique(), columns=["Ürün Kodu"])

writer_btt = pd.ExcelWriter("../reports/btt_eslenik_kodu_olmayan.xlsx", engine = 'xlsxwriter')

btt_eslenik_kod_olmayan_unique.to_excel(writer_btt, sheet_name = 'eslenik_kod_olmayan_sku',index=False)
btt_eslenik_kod_olmayan.to_excel(writer_btt, sheet_name = 'eslenik_kod_olmayan_data',index=False)

writer_btt.save()
writer_btt.close()

In [18]:
horizon_eslenik_kod_olmayan = horizon_saha_df_all[pd.isnull(horizon_saha_df_all["En Güncel Kod"])]
horizon_eslenik_kod_olmayan_unique = pd.DataFrame(horizon_saha_df_all[pd.isnull(horizon_saha_df_all["En Güncel Kod"])]["Ürün Kodu"].unique(), columns=["Ürün Kodu"])

writer_horizon = pd.ExcelWriter("../reports/horizon_eslenik_kodu_olmayan.xlsx", engine = 'xlsxwriter')

horizon_eslenik_kod_olmayan_unique.to_excel(writer_horizon, sheet_name = 'eslenik_kod_olmayan_sku',index=False)
horizon_eslenik_kod_olmayan.to_excel(writer_horizon, sheet_name = 'eslenik_kod_olmayan_data',index=False)

writer_horizon.save()
writer_horizon.close()

## Ürün Eşleme Kodları dosyasında yer almayan kodlar için mevcut ürün kodları verildi.

In [19]:
empty_ = pasifik_df_all[pd.isnull(pasifik_df_all["En Güncel Kod"])].reset_index(drop=True)
full_ = pasifik_df_all[~pd.isnull(pasifik_df_all["En Güncel Kod"])].reset_index(drop=True)
empty_["En Güncel Kod"] = empty_["Ürün Kodu"]
pasifik_df_all = pd.concat([empty_, full_], axis=0, ignore_index=True)
pasifik_df_all = pasifik_df_all.sort_values(pasifik_df_all.columns.to_list()).reset_index(drop=True)

empty_ = btt_df_all[pd.isnull(btt_df_all["En Güncel Kod"])].reset_index(drop=True)
full_ = btt_df_all[~pd.isnull(btt_df_all["En Güncel Kod"])].reset_index(drop=True)
empty_["En Güncel Kod"] = empty_["Ürün Kodu"]
btt_df_all = pd.concat([empty_, full_], axis=0, ignore_index=True)
btt_df_all = btt_df_all.sort_values(btt_df_all.columns.to_list()).reset_index(drop=True)

empty_ = horizon_saha_df_all[pd.isnull(horizon_saha_df_all["En Güncel Kod"])].reset_index(drop=True)
full_ = horizon_saha_df_all[~pd.isnull(horizon_saha_df_all["En Güncel Kod"])].reset_index(drop=True)
empty_["En Güncel Kod"] = empty_["Ürün Kodu"]
horizon_saha_df_all = pd.concat([empty_, full_], axis=0, ignore_index=True)
horizon_saha_df_all = horizon_saha_df_all.sort_values(horizon_saha_df_all.columns.to_list()).reset_index(drop=True)

## Adet adında yeni bir kolon oluşturuldu.

In [20]:
pasifik_df_all["Adet"] = pasifik_df_all["Koli İçi Adet"] * pasifik_df_all["Koli"]
btt_df_all["Adet"] = btt_df_all["Koli İçi Adet"] * btt_df_all["Koli"]
horizon_saha_df_all["Adet"] = horizon_saha_df_all["Koli İçi Adet"] * horizon_saha_df_all["Koli"]

## Delist olan ürünler veriden çıkartıldı.

In [21]:
pasifik_df_all = pasifik_df_all[pasifik_df_all["En Güncel Kod"] != "delist"].reset_index(drop=True)
btt_df_all = btt_df_all[btt_df_all["En Güncel Kod"] != "delist"].reset_index(drop=True)
horizon_saha_df_all = horizon_saha_df_all[horizon_saha_df_all["En Güncel Kod"] != "delist"].reset_index(drop=True)

## Aynı yıl, ay, grup adı, ana kategori adı, kategori adı, marka adı ve SKU kodundaki ürünler için toplam alındı. Sadece Koli İçi Adet için maksimum olan alındı.

---

# Marka adı dahil değil groupby'a
---

In [22]:
dct_to_sum = {"Koli İçi Adet": "sum", "Koli": "sum", "KG": "sum", "TL": "sum", "Adet": "sum"}

pasifik_df_all2 = pasifik_df_all.groupby(["Yıl", "Ay", "Grup adı", "Ana Kategori Adı", "Kategori Adı", "En Güncel Kod"]).agg(dct_to_sum).reset_index()
btt_df_all2 = btt_df_all.groupby(["Yıl", "Ay", "Grup adı", "Ana Kategori Adı", "Kategori Adı", "En Güncel Kod"]).agg(dct_to_sum).reset_index()
horizon_saha_df_all2 = horizon_saha_df_all.groupby(["Yıl", "Ay", "Grup adı", "Ana Kategori Adı", "Kategori Adı", "En Güncel Kod"]).agg(dct_to_sum).reset_index()

In [23]:
pasifik_df_all2["Date"] = pasifik_df_all2["Yıl"].astype(str) + "-" +  pasifik_df_all2["Ay"].astype(str) + "-01"
btt_df_all2["Date"] = btt_df_all2["Yıl"].astype(str) + "-" +  btt_df_all2["Ay"].astype(str) + "-01"
horizon_saha_df_all2["Date"] = horizon_saha_df_all2["Yıl"].astype(str) + "-" +  horizon_saha_df_all2["Ay"].astype(str) + "-01"

pasifik_df_all2["Date"] = pd.to_datetime(pasifik_df_all2["Date"], format="%Y-%m-%d", errors="coerce")
btt_df_all2["Date"] = pd.to_datetime(btt_df_all2["Date"], format="%Y-%m-%d", errors="coerce")
horizon_saha_df_all2["Date"] = pd.to_datetime(horizon_saha_df_all2["Date"], format="%Y-%m-%d", errors="coerce")

## Horizon ve Pasifikte bulunan "Diğer"'lerin yanlarına "_" ile Diğer_Pasifik, Diğer_Horizon yazıldı.

In [24]:
pasifik_df_all2["Grup adı"] = pasifik_df_all2["Grup adı"].apply(lambda x: "Diğer_Pasifik" if x == "Diğer" else x)
horizon_saha_df_all2["Grup adı"] = horizon_saha_df_all2["Grup adı"].apply(lambda x: "Diğer_Horizon" if x == "Diğer" else x)

# Kontrol: Herhangi bir SKU için koli içi adeti birden fazla geldi mi?

---

In [25]:
pasifik_temp = pasifik_df_all.copy()
btt_temp = btt_df_all.copy()
horizon_temp = horizon_saha_df_all.copy()

pasifik_temp["Grup adı"] = pasifik_temp["Grup adı"].apply(lambda x: "Diğer_Pasifik" if x == "Diğer" else x)
horizon_temp["Grup adı"] = horizon_temp["Grup adı"].apply(lambda x: "Diğer_Horizon" if x == "Diğer" else x)

In [26]:
unique = pd.concat([pasifik_temp, btt_temp, horizon_temp], axis=0, ignore_index=True)

koli_ici_fark = pd.concat([unique.groupby(["En Güncel Kod", "Grup adı"]).agg({"Koli İçi Adet": "min"}), unique.groupby(["En Güncel Kod", "Grup adı"]).agg({"Koli İçi Adet": "max"})], axis=1)
koli_ici_fark.columns = ["min", "max"]
koli_ici_fark["Fark"] = koli_ici_fark["max"] - koli_ici_fark["min"]
koli_ici_fark.reset_index(inplace=True)
koli_ici_fark[koli_ici_fark["Fark"] != 0].to_excel("../reports/ayni_sku_kod_farkli_koli_adedi.xlsx", index=False)

# Kontrol: Güncel Koda denk gelen eski ürün isimleri neler?

In [27]:
unique_pasifik = pasifik_df_all[["En Güncel Kod", "Ürün Adı"]]
unique_btt = btt_df_all[["En Güncel Kod", "Ürün Adı"]]
unique_horizon = horizon_saha_df_all[["En Güncel Kod", "Ürün Adı"]]

unique_pasifik = unique_pasifik.drop_duplicates(subset=["En Güncel Kod", "Ürün Adı"]).reset_index(drop=True)
unique_btt = unique_btt.drop_duplicates(subset=["En Güncel Kod", "Ürün Adı"]).reset_index(drop=True)
unique_horizon = unique_horizon.drop_duplicates(subset=["En Güncel Kod", "Ürün Adı"]).reset_index(drop=True)

unique_pasifik.groupby("En Güncel Kod")["Ürün Adı"].agg(lambda x: "-".join(x)).to_excel("../reports/pasifik_guncel_kod_karsiligi_urunler.xlsx", index=True)
unique_btt.groupby("En Güncel Kod")["Ürün Adı"].agg(lambda x: "-".join(x)).to_excel("../reports/btt_guncel_kod_karsiligi_urunler.xlsx", index=True)
unique_horizon.groupby("En Güncel Kod")["Ürün Adı"].agg(lambda x: "-".join(x)).to_excel("../reports/horizon_guncel_kod_karsiligi_urunler.xlsx", index=True)

---

---

# SKU bazında gözlem sayısı

In [28]:
num_pasifik = pasifik_df_all2.groupby(["En Güncel Kod", "Grup adı"]).count()["Yıl"].reset_index().rename(columns={"Yıl": "Count"})
num_btt = btt_df_all2.groupby(["En Güncel Kod", "Grup adı"]).count()["Yıl"].reset_index().rename(columns={"Yıl": "Count"})
num_horizon = horizon_saha_df_all2.groupby(["En Güncel Kod", "Grup adı"]).count()["Yıl"].reset_index().rename(columns={"Yıl": "Count"})

In [29]:
num_all = pd.concat([num_btt, num_horizon, num_pasifik], ignore_index=True, axis=0)
num_all.sort_values(by=["En Güncel Kod", "Grup adı", "Count"], inplace=True)
num_all.to_excel("../reports/sku_bazinda_sayilar.xlsx", index=False)

---

---
# Missing Value'lara 1 atanması (0 atanmamasının sebebi, prediction yapılırken MAPE'nin infinitive gelmesi)
---

---
# Pasifik Filling Missing Values
---

In [30]:
pasifik_urun_isimleri = pasifik_df_all[["Marka Adı", "Ürün Adı", "En Güncel Kod"]].drop_duplicates(subset=["Marka Adı", "En Güncel Kod"],ignore_index=True,keep="first")
pasifik_urun_isimleri = pasifik_urun_isimleri[~((pasifik_urun_isimleri["Marka Adı"] == "DANKEK BATON") & (pasifik_urun_isimleri["En Güncel Kod"] == 80403))]
pasifik_df_all2 = pd.merge(pasifik_df_all2, pasifik_urun_isimleri[["En Güncel Kod", "Marka Adı", "Ürün Adı"]].drop_duplicates(subset=["En Güncel Kod", "Marka Adı", "Ürün Adı"], keep="first"), on="En Güncel Kod", how="left")
pasifik_df_all2 = pasifik_df_all2[pasifik_df_all2.columns.to_list()[:5]+pasifik_df_all2.columns.to_list()[-2:]+[pasifik_df_all2.columns.to_list()[5]]+pasifik_df_all2.columns.to_list()[6:12]]



horizon_urun_isimleri = horizon_saha_df_all[["Marka Adı", "Ürün Adı", "En Güncel Kod"]].drop_duplicates(subset=["Marka Adı", "En Güncel Kod"],ignore_index=True,keep="first")
horizon_urun_isimleri = horizon_urun_isimleri[~(((horizon_urun_isimleri["Marka Adı"] == "DANKEK BATON") & (horizon_urun_isimleri["En Güncel Kod"] == 80403)) | 
                                                ((horizon_urun_isimleri["Marka Adı"] == "MAVİ YEŞİL") & (horizon_urun_isimleri["En Güncel Kod"] == 11802)) |
                                                ((horizon_urun_isimleri["Marka Adı"] == "MAVİ YEŞİL") & (horizon_urun_isimleri["En Güncel Kod"] == 74306)) |
                                                ((horizon_urun_isimleri["Marka Adı"] == "AS KRAKER") & (horizon_urun_isimleri["En Güncel Kod"] == 190502)))]
horizon_saha_df_all2 = pd.merge(horizon_saha_df_all2, horizon_urun_isimleri[["En Güncel Kod", "Marka Adı", "Ürün Adı"]], on="En Güncel Kod", how="left")
horizon_saha_df_all2 = horizon_saha_df_all2[horizon_saha_df_all2.columns.to_list()[:5]+horizon_saha_df_all2.columns.to_list()[-2:]+[horizon_saha_df_all2.columns.to_list()[5]]+horizon_saha_df_all2.columns.to_list()[6:12]]



btt_urun_isimleri = btt_df_all[["Marka Adı", "Ürün Adı", "En Güncel Kod"]].drop_duplicates(subset=["Marka Adı", "En Güncel Kod"],ignore_index=True,keep="first")
btt_urun_isimleri = btt_urun_isimleri[~((btt_urun_isimleri["Marka Adı"] == "DANKEK BATON") & (btt_urun_isimleri["En Güncel Kod"] == 80403))]
btt_df_all2 = pd.merge(btt_df_all2, btt_urun_isimleri[["En Güncel Kod", "Marka Adı", "Ürün Adı"]].drop_duplicates(subset=["En Güncel Kod", "Marka Adı", "Ürün Adı"], keep="first"), on="En Güncel Kod", how="left")
btt_df_all2 = btt_df_all2[btt_df_all2.columns.to_list()[:5]+btt_df_all2.columns.to_list()[-2:]+[btt_df_all2.columns.to_list()[5]]+btt_df_all2.columns.to_list()[6:12]]

In [31]:
pasifik_df_all_filled = []
for idx, test in pasifik_df_all2.groupby(["En Güncel Kod", "Yıl", "Grup adı"]):
    test.reset_index(drop=True, inplace=True)
    for i in range(1, 13):
        try:
            if i == test.loc[i-1, "Ay"]:
                if i == 13:
                    break
            else:
                test.loc[-1] = test.loc[0]
                test.loc[-1, "Ay"], test.loc[-1, "Koli İçi Adet"], test.loc[-1, "Koli"],  \
                test.loc[-1, "KG"], test.loc[-1, "TL"], test.loc[-1, "Adet"], test.loc[-1, "Date"] = i, 0, 0, 0, 0, 1, str(test.loc[-1, "Yıl"])+"-"+str(i)+"-"+str("01")
                test = test.sort_values(by=["Yıl", "Ay"]).reset_index(drop=True)
        except:
            test.loc[-1] = test.loc[0]
            test.loc[-1, "Ay"], test.loc[-1, "Koli İçi Adet"], test.loc[-1, "Koli"],  \
            test.loc[-1, "KG"], test.loc[-1, "TL"], test.loc[-1, "Adet"], test.loc[-1, "Date"] = i, 0, 0, 0, 0, 1, str(test.loc[-1, "Yıl"])+"-"+str(i)+"-"+str("01")
            test = test.sort_values(by=["Yıl", "Ay"]).reset_index(drop=True)
        test["Date"] = pd.to_datetime(test["Date"], format="%Y-%m-%d", errors="coerce")
    rows_to_drop = []
    start = test.index[0]
    length = 1
    while (test.loc[start, "Adet"] == 1) and (length < len(test)):
        rows_to_drop.append(start)
        length+=1
        start+=1
    test.drop(index=rows_to_drop, inplace=True)
    pasifik_df_all_filled.append(test)

In [32]:
pasifik_df_all2 = pd.concat(pasifik_df_all_filled, ignore_index=True)

---
# Horizon Filling Missing Values
---

In [33]:
horizon_saha_df_all_filled = []
for idx, test in horizon_saha_df_all2.groupby(["En Güncel Kod", "Yıl", "Grup adı"]):
    test.reset_index(drop=True, inplace=True)
    for i in range(1, 13):
        try:
            if i == test.loc[i-1, "Ay"]:
                if i == 13:
                    break
            else:
                test.loc[-1] = test.loc[0]
                test.loc[-1, "Ay"], test.loc[-1, "Koli İçi Adet"], test.loc[-1, "Koli"],  \
                test.loc[-1, "KG"], test.loc[-1, "TL"], test.loc[-1, "Adet"], test.loc[-1, "Date"] = i, 0, 0, 0, 0, 1, str(test.loc[-1, "Yıl"])+"-"+str(i)+"-"+str("01")
                test = test.sort_values(by=["Yıl", "Ay"]).reset_index(drop=True)
        except:
            test.loc[-1] = test.loc[0]
            test.loc[-1, "Ay"], test.loc[-1, "Koli İçi Adet"], test.loc[-1, "Koli"],  \
            test.loc[-1, "KG"], test.loc[-1, "TL"], test.loc[-1, "Adet"], test.loc[-1, "Date"] = i, 0, 0, 0, 0, 1, str(test.loc[-1, "Yıl"])+"-"+str(i)+"-"+str("01")
            test = test.sort_values(by=["Yıl", "Ay"]).reset_index(drop=True)
        test["Date"] = pd.to_datetime(test["Date"], format="%Y-%m-%d", errors="coerce")
    
    rows_to_drop = []
    start = test.index[0]
    length = 1
    while (test.loc[start, "Adet"] == 1) and (length < len(test)):
        rows_to_drop.append(start)
        length+=1
        start+=1
    test.drop(index=rows_to_drop, inplace=True)
    horizon_saha_df_all_filled.append(test)

In [34]:
horizon_saha_df_all2 = pd.concat(horizon_saha_df_all_filled, ignore_index=True)

---
# BTT Filling Missing Values
---

In [ ]:
btt_df_all_filled = []
for idx, test in btt_df_all2.groupby(["En Güncel Kod", "Yıl", "Grup adı"]):
    test.reset_index(drop=True, inplace=True)
    for i in range(1, 13):
        try:
            if i == test.loc[i-1, "Ay"]:
                if i == 13:
                    break
            else:
                test.loc[-1] = test.loc[0]
                test.loc[-1, "Ay"], test.loc[-1, "Koli İçi Adet"], test.loc[-1, "Koli"],  \
                test.loc[-1, "KG"], test.loc[-1, "TL"], test.loc[-1, "Adet"], test.loc[-1, "Date"] = i, 0, 0, 0, 0, 1, str(test.loc[-1, "Yıl"])+"-"+str(i)+"-"+str("01")
                test = test.sort_values(by=["Yıl", "Ay"]).reset_index(drop=True)
        except:
            test.loc[-1] = test.loc[0]
            test.loc[-1, "Ay"], test.loc[-1, "Koli İçi Adet"], test.loc[-1, "Koli"],  \
            test.loc[-1, "KG"], test.loc[-1, "TL"], test.loc[-1, "Adet"], test.loc[-1, "Date"] = i, 0, 0, 0, 0, 1, str(test.loc[-1, "Yıl"])+"-"+str(i)+"-"+str("01")
            test = test.sort_values(by=["Yıl", "Ay"]).reset_index(drop=True)
        test["Date"] = pd.to_datetime(test["Date"], format="%Y-%m-%d", errors="coerce")

    rows_to_drop = []
    start = test.index[0]
    length = 1
    while (test.loc[start, "Adet"] == 1) and (length < len(test)):
        rows_to_drop.append(start)
        length+=1
        start+=1
    test.drop(index=rows_to_drop, inplace=True)
    btt_df_all_filled.append(test)

In [ ]:
btt_df_all2 = pd.concat(btt_df_all_filled, ignore_index=True)

---

---

# Aktivite Datası İçin Ürün Kod Eşleme

---

---

# Pasifik Aktivite

## Left join ile güncel kodlar getirildi. Delist olan ürünler listeden çıkartıldı. "Çeyrek" sütunu silindi. En güncel kod sütunnuda bulunamayan değerler Ürün Kodu sütunundan çekildi.

In [ ]:
pasifik_aktivite_df2 = pd.merge(pasifik_aktivite_df, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")
pasifik_aktivite_df2 = pasifik_aktivite_df2[pasifik_aktivite_df2["En Güncel Kod"] != "delist"].reset_index(drop=True)
pasifik_aktivite_df2.drop("Çeyrek", inplace=True, axis=1)
pasifik_aktivite_df2['En Güncel Kod'] = pasifik_aktivite_df2['En Güncel Kod'].fillna(pasifik_aktivite_df2['Ürün Kodu'])
pasifik_aktivite_df2.drop(columns="Ürün Kodu", axis=1, inplace=True)
pasifik_aktivite_df2.to_csv("../data/_pas_akt_data.csv", index=False)

## Hangi güncel ürün kodundan kaç tane girildiğinin kontrolü

## Duplike durumdaki ürün kodlarının excel olarak dışarıya alınması

In [ ]:
x = pasifik_aktivite_df2.columns.to_list()[:5]
y = pasifik_aktivite_df2.columns.to_list()[-1]
x.append(y)
pasifik_aktivite_df2[pasifik_aktivite_df2[x].duplicated(keep=False)].sort_values(by=["En Güncel Kod", "Yıl", "Ay", "Müşteri Grup", "Ana Kategori Adı", "Kategori Adı"]).to_excel("../reports/duplike_pasifik_aktivite_datasi.xlsx", index=False)

---
## Aynı anda farklı iki aktivite tipi var mı?
---

In [ ]:
# pa_df = pasifik aktivite df (sadece aynı ay farklı aktivite tipini görmek için)
pa_df = pasifik_aktivite_df2.copy()

pa_df.drop(columns=["Ana Kategori Adı", "Kategori Adı", "Marka Adı", "Ürün Adı"], axis=1, inplace=True)
pa_df.drop_duplicates(subset=pa_df.columns.to_list(), inplace=True, ignore_index=True)
check_pa_df = pa_df.pivot_table(index=["Yıl", "Ay", "En Güncel Kod"], columns="Aktivite Tipi", aggfunc=lambda x: len(x.unique())).sum(axis=1).reset_index()
check_pa_df = check_pa_df[check_pa_df[0] > 1].reset_index(drop=True)

---

---

---

# Pasifik Aktivite Ciro - Promosyon Tutarı ve İskonto Tekilleştirme (ORTALAMA ALARAK)

In [ ]:
ort_pas = {"Raf Tavsiye Satış Fiyatı": "mean", "İndirimli Raf Satış Fiyatı": "mean", "İndirim %": "mean", "Aktivite Tipi": "first"}
pasifik_aktivite_df3 = pasifik_aktivite_df2.groupby(["En Güncel Kod", "Yıl", "Ay", "Müşteri Grup"]).agg(ort_pas).reset_index()
pasifik_aktivite_df3 = pd.merge(pasifik_aktivite_df3, pasifik_aktivite_df2[["Yıl", "Ay", "Müşteri Grup", "En Güncel Kod", 
                                                 "Ana Kategori Adı", "Kategori Adı", "Marka Adı"]],
                           how="left", 
                           on=["En Güncel Kod", "Yıl", "Ay", "Müşteri Grup"])

pasifik_aktivite_df3.drop_duplicates(subset=pasifik_aktivite_df3.columns.to_list(), inplace=True)

In [ ]:
pasifik_aktivite_df3.reset_index(drop=True, inplace=True)
pasifik_aktivite_df3 = pasifik_aktivite_df3[pasifik_aktivite_df2.drop("Ürün Adı", axis=1).columns.to_list()]

In [ ]:
x = pasifik_aktivite_df3.columns.to_list()[:5]
y = pasifik_aktivite_df3.columns.to_list()[-2:]
x.extend(y)
#pasifik_aktivite_df3[pasifik_aktivite_df3[x].duplicated(keep=False)].sort_values(by=["En Güncel Kod", "Yıl", "Ay", "Müşteri Grup", "Ana Kategori Adı", "Kategori Adı"]).to_excel("../reports/duplike_pasifik_aktivite_datasi.xlsx", index=False)

---

---

---

# Horizon Aktivite

In [ ]:
saha_aktivite_detay2 = pd.merge(saha_aktivite_detay, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")
saha_aktivite_detay2 = saha_aktivite_detay2[saha_aktivite_detay2["En Güncel Kod"] != "delist"].reset_index(drop=True)
saha_aktivite_detay2.drop("Çeyrek", inplace=True, axis=1)
saha_aktivite_detay2['En Güncel Kod'] = saha_aktivite_detay2['En Güncel Kod'].fillna(saha_aktivite_detay2['Ürün Kodu'])
saha_aktivite_detay2.drop(columns="Ürün Kodu", axis=1, inplace=True)
saha_aktivite_detay2["İskonto %"].replace("#DIV/0", np.nan,inplace=True)
saha_aktivite_detay2.to_csv("../data/_saha_akt_data.csv", index=False)

In [ ]:
x = saha_aktivite_detay2.columns.to_list()[:5]
y = saha_aktivite_detay2.columns.to_list()[-1]
x.append(y)
saha_aktivite_detay2[saha_aktivite_detay2[x].duplicated(keep=False)].sort_values(by=["En Güncel Kod", "Yıl", "Ay", "Saha Müşteri Grup", "Ana Kategori Adı", "Kategori Adı"]).to_excel("../reports/duplike_saha_aktivite_datasi.xlsx", index=False)

# Horizon Aktivite Ciro - Promosyon Tutarı ve İskonto Tekilleştirme (ORTALAMA ALARAK)

In [ ]:
ort = {"Ciro (Kull. İade Düş.)": "mean", "Promosyon Tutarı": "mean", "İskonto %": "mean"}
saha_aktivite_detay3 = saha_aktivite_detay2.groupby(["En Güncel Kod", "Yıl", "Ay", "Saha Müşteri Grup"]).agg(ort).reset_index()

saha_aktivite_detay3 = pd.merge(saha_aktivite_detay3, saha_aktivite_detay2[["Yıl", "Ay", "Saha Müşteri Grup", "En Güncel Kod", 
                                                 "Ana Kategori Adı", "Kategori Adı", "Marka Adı"]],
                           how="left", 
                           on=["En Güncel Kod", "Yıl", "Ay", "Saha Müşteri Grup"])

saha_aktivite_detay3.drop_duplicates(subset=saha_aktivite_detay3.columns.to_list(), inplace=True)

In [ ]:
saha_aktivite_detay3 = saha_aktivite_detay3[saha_aktivite_detay2.drop("Ürün Adı (Mobis)", axis=1).columns.to_list()].reset_index(drop=True)

In [ ]:
x = saha_aktivite_detay3.columns.to_list()[:5]
y = saha_aktivite_detay3.columns.to_list()[-1]
x.append(y)
#saha_aktivite_detay3[saha_aktivite_detay3[x].duplicated(keep=False)].sort_values(by=["En Güncel Kod", "Yıl", "Ay", "Saha Müşteri Grup", "Ana Kategori Adı", "Kategori Adı"]).to_excel("../reports/duplike_saha_aktivite_datasi.xlsx", index=False)

In [ ]:
saha_aktivite_detay3.rename(columns={"Saha Müşteri Grup": "Grup adı"}, inplace=True)
saha_aktivite_detay3["Grup adı"] = saha_aktivite_detay3["Grup adı"].apply(lambda x: "Diğer_Horizon" if x == "Diğer" else x)

---

---

# Fiyat Listesi

## Horizon Fiyatları
---

In [ ]:
fiyat_lst_horizon.drop_duplicates(subset=fiyat_lst_horizon.columns.to_list(), keep="first", ignore_index=True, inplace=True)

In [ ]:
fiyat_lst_horizon["Malzeme"] = fiyat_lst_horizon["Malzeme"].str.replace("-", "")
fiyat_lst_horizon["Malzeme"] = fiyat_lst_horizon["Malzeme"].astype(int)*1

In [ ]:
fiyat_lst_horizon_df = fiyat_lst_horizon.copy()

In [ ]:
fiyat_lst_horizon_df["Baslangic_Yıl"] = fiyat_lst_horizon_df["Bşl.tarihi"].apply(lambda x: x.year)
fiyat_lst_horizon_df["Baslangic_Ay"] = fiyat_lst_horizon_df["Bşl.tarihi"].apply(lambda x: x.month)
fiyat_lst_horizon_df["Baslangic_Gun"] = fiyat_lst_horizon_df["Bşl.tarihi"].apply(lambda x: x.day)
fiyat_lst_horizon_df["Gecerlilik_Yıl"] = fiyat_lst_horizon_df["Gçrl.sonu"].apply(lambda x: x.year)
fiyat_lst_horizon_df["Gecerlilik_Ay"] = fiyat_lst_horizon_df["Gçrl.sonu"].apply(lambda x: x.month)
fiyat_lst_horizon_df["Gecerlilik_Gun"] = fiyat_lst_horizon_df["Gçrl.sonu"].apply(lambda x: x.day)

In [ ]:
fiyat_lst_horizon_df["Baslangic_Yıl"] = fiyat_lst_horizon_df["Baslangic_Yıl"].apply(lambda x: 2022 if x > 2021 else x)
fiyat_lst_horizon_df["Gecerlilik_Yıl"] = fiyat_lst_horizon_df["Gecerlilik_Yıl"].apply(lambda x: 2022 if x > 2021 else x)

In [ ]:
time_index = horizon_saha_df_all2["Date"].unique()

In [ ]:
h_fiyat_unique = []

for malzeme in fiyat_lst_horizon_df["Malzeme"].unique():
    temp_time_df = pd.DataFrame({"Fiyat": [np.nan]}, index=time_index)
    temp_time_df = temp_time_df.reset_index().rename(columns={"index":"date"})    
    temp_time_df["En Güncel Kod"] = malzeme
    temp_time_df["fiyat_gecisi"] = 0
    malzeme_df = fiyat_lst_horizon_df[fiyat_lst_horizon_df["Malzeme"] == malzeme].reset_index(drop=True)
    malzeme_df.drop(columns=["KşTü", "Koşul türü", "Tanım", "Ana Kategori", "Kategori", "ÖB"], axis=1, inplace=True)
    malzeme_df.drop_duplicates(subset=malzeme_df.columns.to_list(), inplace=True, ignore_index=True)
    malzeme_df.sort_values(by=["Baslangic_Yıl", "Baslangic_Ay", "Baslangic_Gun"], ignore_index=True, inplace=True)
    check_idx1 = []
    if len(malzeme_df) > 1:
        for row1 in malzeme_df.index:
            for row2 in malzeme_df[row1+1:].index:
                if (malzeme_df.loc[row1]["Gecerlilik_Yıl"] == malzeme_df.loc[row2]["Baslangic_Yıl"]) and (malzeme_df.loc[row1]["Gecerlilik_Ay"] == malzeme_df.loc[row2]["Baslangic_Ay"]):
                    num_days = calendar.monthrange(int(malzeme_df.loc[row2]["Baslangic_Yıl"]), int(malzeme_df.loc[row2]["Baslangic_Ay"]))[1]
                    fyt=((int(malzeme_df.loc[row1]["Gecerlilik_Gun"])*malzeme_df.loc[row1]["     Tutar"]) + (num_days - int(malzeme_df.loc[row2]["Baslangic_Gun"]) + 1)*malzeme_df.loc[row2]["     Tutar"])/num_days

                    end_idx1 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx1 = temp_time_df[temp_time_df["date"] == end_idx1].index
                    temp_time_df.loc[final_idx1, "Fiyat"] = fyt
                    temp_time_df.loc[final_idx1,"fiyat_gecisi"] = 1

                elif (malzeme_df.loc[row1, "Gecerlilik_Gun"] == calendar.monthrange(int(malzeme_df.loc[row1]["Gecerlilik_Yıl"]), int(malzeme_df.loc[row1]["Gecerlilik_Ay"]))[1] \
                     and malzeme_df.loc[row2, "Baslangic_Gun"] == 1):
                    fyt5=malzeme_df.loc[row1]["     Tutar"]
                    fyt6=malzeme_df.loc[row2]["     Tutar"]
                    end_idx5 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    end_idx6 =  datetime(int(malzeme_df.loc[row2, "Baslangic_Yıl"]), int(malzeme_df.loc[row2, "Baslangic_Ay"]), 1)
                    final_idx5 = temp_time_df[temp_time_df["date"] == end_idx5].index
                    final_idx6 = temp_time_df[temp_time_df["date"] == end_idx6].index
                    temp_time_df.loc[final_idx5, "Fiyat"] = fyt5
                    temp_time_df.loc[final_idx6, "Fiyat"] = fyt6

                else:
                    fyt2=malzeme_df.loc[row1]["     Tutar"]
                    start_idx2 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                    end_idx2 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx2 = temp_time_df[(temp_time_df["date"] > start_idx2) & (temp_time_df["date"] < end_idx2)].index
                    temp_time_df.loc[final_idx2, "Fiyat"] = fyt2
            if (row1 == len(malzeme_df)-1) or (row1 == len(malzeme_df)-2):
                fyt3=malzeme_df.loc[row1]["     Tutar"]
                start_idx3 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                end_idx3 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                final_idx3 = temp_time_df[(temp_time_df["date"] > start_idx3) & (temp_time_df["date"] < end_idx3)].index
                temp_time_df.loc[final_idx3, "Fiyat"] = fyt3

    else:
        for row1 in malzeme_df.index:
            fyt4=malzeme_df.loc[row1]["     Tutar"]
            start_idx4 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
            end_idx4 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
            final_idx4 = temp_time_df[(temp_time_df["date"] >= start_idx4) & (temp_time_df["date"] <= end_idx4)].index
            temp_time_df.loc[final_idx4, "Fiyat"] = fyt4

    if (malzeme_df.loc[0, "Baslangic_Yıl"] >= min(horizon_saha_df_all2["Yıl"].unique())) and (len(malzeme_df) > 1):
        temp_time_df.loc[temp_time_df[~pd.isnull(temp_time_df["Fiyat"])].index[0]-1, "Fiyat"] = malzeme_df.loc[0, "     Tutar"]
    temp_time_df = temp_time_df.dropna().reset_index(drop=True)

    h_fiyat_unique.append(temp_time_df)

In [ ]:
h_fiyat_unique = pd.concat(h_fiyat_unique)
h_fiyat_unique.reset_index(drop=True, inplace=True)

h_fiyat_unique.rename(columns={"En Güncel Kod": "Ürün Kodu", "date": "Date"}, inplace=True)
h_fiyat_unique = h_fiyat_unique.merge(eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left")
h_fiyat_unique["En Güncel Kod"].fillna(h_fiyat_unique["Ürün Kodu"], inplace=True)

# Fiyatların en güncel kod ile check edilmesi

In [ ]:
h_fiyat_unique = h_fiyat_unique[h_fiyat_unique["En Güncel Kod"] != "delist"].reset_index(drop=True)

h_fiyat_unique = h_fiyat_unique.sort_values(by=["En Güncel Kod", "Date"]).reset_index(drop=True)
drop_rows_h = h_fiyat_unique[h_fiyat_unique[["Date", "En Güncel Kod", "Fiyat"]].duplicated(keep=False)].index

export_h = h_fiyat_unique[~h_fiyat_unique.index.isin(drop_rows_h)].reset_index(drop=True)
export_h[export_h[["En Güncel Kod", "Date"]].duplicated(keep=False)].to_excel("../reports/horizon_fiyat_eslestirme.xlsx", index=False)

In [ ]:
h_fiyat_unique = h_fiyat_unique.drop(columns="Ürün Kodu", axis=1)
# Aynı aya denk gelen ürünlerin fiyatlarının ortalaması alınıp, herhangi birinde fiyat geçişi varsa 1 alınır.
h_fiyat_unique = h_fiyat_unique.groupby(["Date", "En Güncel Kod"]).agg({"Fiyat": "mean", "fiyat_gecisi": "max"}).reset_index()

## Pasifik Fiyatları
---

In [ ]:
fiyat_lst_pasifik.drop_duplicates(subset=fiyat_lst_pasifik.columns.to_list(), keep="first", ignore_index=True, inplace=True)

In [ ]:
fiyat_lst_pasifik["Malzeme"] = fiyat_lst_pasifik["Malzeme"].str.replace("-", "")
fiyat_lst_pasifik["Malzeme"] = fiyat_lst_pasifik["Malzeme"].astype(int)*1

In [ ]:
fiyat_lst_pasifik_df = fiyat_lst_pasifik.copy()

In [ ]:
fiyat_lst_pasifik_df["Baslangic_Yıl"] = fiyat_lst_pasifik_df["Bşl.tarihi"].apply(lambda x: x.year)
fiyat_lst_pasifik_df["Baslangic_Ay"] = fiyat_lst_pasifik_df["Bşl.tarihi"].apply(lambda x: x.month)
fiyat_lst_pasifik_df["Baslangic_Gun"] = fiyat_lst_pasifik_df["Bşl.tarihi"].apply(lambda x: x.day)
fiyat_lst_pasifik_df["Gecerlilik_Yıl"] = fiyat_lst_pasifik_df["Gçrl.sonu"].apply(lambda x: x.year)
fiyat_lst_pasifik_df["Gecerlilik_Ay"] = fiyat_lst_pasifik_df["Gçrl.sonu"].apply(lambda x: x.month)
fiyat_lst_pasifik_df["Gecerlilik_Gun"] = fiyat_lst_pasifik_df["Gçrl.sonu"].apply(lambda x: x.day)

In [ ]:
fiyat_lst_pasifik_df["Baslangic_Yıl"] = fiyat_lst_pasifik_df["Baslangic_Yıl"].apply(lambda x: 2022 if x > 2021 else x)
fiyat_lst_pasifik_df["Gecerlilik_Yıl"] = fiyat_lst_pasifik_df["Gecerlilik_Yıl"].apply(lambda x: 2022 if x > 2021 else x)

In [ ]:
time_index = pasifik_df_all2["Date"].unique()

In [ ]:
p_fiyat_unique = []

for malzeme in fiyat_lst_pasifik_df["Malzeme"].unique():
    temp_time_df = pd.DataFrame({"Fiyat": [np.nan]}, index=time_index)
    temp_time_df = temp_time_df.reset_index().rename(columns={"index":"date"})    
    temp_time_df["En Güncel Kod"] = malzeme
    temp_time_df["fiyat_gecisi"] = 0
    malzeme_df = fiyat_lst_pasifik_df[fiyat_lst_pasifik_df["Malzeme"] == malzeme].reset_index(drop=True)
    malzeme_df.drop(columns=["KşTü", "KşTü.1", "Malzeme Tanım", "Ana Kategori", "Kategori"], axis=1, inplace=True)
    malzeme_df.drop_duplicates(subset=malzeme_df.columns.to_list(), inplace=True, ignore_index=True)
    malzeme_df.sort_values(by=["Baslangic_Yıl", "Baslangic_Ay", "Baslangic_Gun"], ignore_index=True, inplace=True)
    check_idx1 = []
    if len(malzeme_df) > 1:
        for row1 in malzeme_df.index:
            for row2 in malzeme_df[row1+1:].index:
                if (malzeme_df.loc[row1]["Gecerlilik_Yıl"] == malzeme_df.loc[row2]["Baslangic_Yıl"]) and (malzeme_df.loc[row1]["Gecerlilik_Ay"] == malzeme_df.loc[row2]["Baslangic_Ay"]):
                    num_days = calendar.monthrange(int(malzeme_df.loc[row2]["Baslangic_Yıl"]), int(malzeme_df.loc[row2]["Baslangic_Ay"]))[1]
                    fyt=((int(malzeme_df.loc[row1]["Gecerlilik_Gun"])*malzeme_df.loc[row1]["Koli TL"]) + (num_days - int(malzeme_df.loc[row2]["Baslangic_Gun"])+1)*malzeme_df.loc[row2]["Koli TL"])/num_days

                    end_idx1 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx1 = temp_time_df[temp_time_df["date"] == end_idx1].index
                    temp_time_df.loc[final_idx1, "Fiyat"] = fyt
                    temp_time_df.loc[final_idx1,"fiyat_gecisi"] = 1

                elif (malzeme_df.loc[row1, "Gecerlilik_Gun"] == calendar.monthrange(int(malzeme_df.loc[row1]["Gecerlilik_Yıl"]), int(malzeme_df.loc[row1]["Gecerlilik_Ay"]))[1] \
                     and malzeme_df.loc[row2, "Baslangic_Gun"] == 1):
                    fyt5=malzeme_df.loc[row1]["Koli TL"]
                    fyt6=malzeme_df.loc[row2]["Koli TL"]
                    end_idx5 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    end_idx6 =  datetime(int(malzeme_df.loc[row2, "Baslangic_Yıl"]), int(malzeme_df.loc[row2, "Baslangic_Ay"]), 1)
                    final_idx5 = temp_time_df[temp_time_df["date"] == end_idx5].index
                    final_idx6 = temp_time_df[temp_time_df["date"] == end_idx6].index
                    temp_time_df.loc[final_idx5, "Fiyat"] = fyt5
                    temp_time_df.loc[final_idx6, "Fiyat"] = fyt6


                else:
                    if malzeme_df.loc[row1, "Baslangic_Gun"] != 1:
                        fyt2=malzeme_df.loc[row1]["Koli TL"]
                        start_idx2 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                        end_idx2 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                        final_idx2 = temp_time_df[(temp_time_df["date"] > start_idx2) & (temp_time_df["date"] < end_idx2)].index
                        temp_time_df.loc[final_idx2, "Fiyat"] = fyt2
                    else:
                        fyt2=malzeme_df.loc[row1]["Koli TL"]
                        start_idx2 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                        end_idx2 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                        final_idx2 = temp_time_df[(temp_time_df["date"] >= start_idx2) & (temp_time_df["date"] < end_idx2)].index
                        temp_time_df.loc[final_idx2, "Fiyat"] = fyt2
                        
            if (row1 == len(malzeme_df)-1) or (row1 == len(malzeme_df)-2):
                if malzeme_df.loc[row1, "Baslangic_Gun"] != 1:
                    fyt3=malzeme_df.loc[row1]["Koli TL"]
                    start_idx3 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                    end_idx3 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx3 = temp_time_df[(temp_time_df["date"] > start_idx3) & (temp_time_df["date"] < end_idx3)].index
                    temp_time_df.loc[final_idx3, "Fiyat"] = fyt3
                else:
                    fyt3=malzeme_df.loc[row1]["Koli TL"]
                    start_idx3 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                    end_idx3 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx3 = temp_time_df[(temp_time_df["date"] >= start_idx3) & (temp_time_df["date"] < end_idx3)].index
                    temp_time_df.loc[final_idx3, "Fiyat"] = fyt3
    else:
        for row1 in malzeme_df.index:
            fyt4=malzeme_df.loc[row1]["Koli TL"]
            start_idx4 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
            end_idx4 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
            final_idx4 = temp_time_df[(temp_time_df["date"] >= start_idx4) & (temp_time_df["date"] <= end_idx4)].index
            temp_time_df.loc[final_idx4, "Fiyat"] = fyt4
    
    if (malzeme_df.loc[0, "Baslangic_Yıl"] >= min(pasifik_df_all2["Yıl"].unique())) and (len(malzeme_df) > 1):
        temp_time_df.loc[temp_time_df[~pd.isnull(temp_time_df["Fiyat"])].index[0]-1, "Fiyat"] = malzeme_df.loc[0, "Koli TL"]
    temp_time_df = temp_time_df.dropna().reset_index(drop=True)
        
    p_fiyat_unique.append(temp_time_df)

In [ ]:
p_fiyat_unique = pd.concat(p_fiyat_unique)
p_fiyat_unique.reset_index(drop=True, inplace=True)

p_fiyat_unique.rename(columns={"En Güncel Kod": "Ürün Kodu", "date": "Date"}, inplace=True)
p_fiyat_unique = p_fiyat_unique.merge(eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left")
p_fiyat_unique["En Güncel Kod"].fillna(p_fiyat_unique["Ürün Kodu"], inplace=True)

# p_fiyat_uniqueların en güncel kod ile check edilmesi

In [ ]:
p_fiyat_unique = p_fiyat_unique[p_fiyat_unique["En Güncel Kod"] != "delist"].reset_index(drop=True)

p_fiyat_unique = p_fiyat_unique.sort_values(by=["En Güncel Kod", "Date"]).reset_index(drop=True)
drop_rows_p = p_fiyat_unique[p_fiyat_unique[["Date", "En Güncel Kod", "Fiyat"]].duplicated(keep=False)].index

export_p = p_fiyat_unique[~p_fiyat_unique.index.isin(drop_rows_p)].reset_index(drop=True)
export_p[export_p[["En Güncel Kod", "Date"]].duplicated(keep=False)].to_excel("../reports/pasifik_fiyat_eslestirme.xlsx", index=False)

In [ ]:
# Dosya ile karşılaştır. Ortalama almanı gerektirecek durumları düşün.

In [ ]:
p_fiyat_unique = p_fiyat_unique.drop(columns="Ürün Kodu", axis=1)
# Aynı aya denk gelen ürünlerin fiyatlarının ortalaması alınıp, herhangi birinde fiyat geçişi varsa 1 alınır.
p_fiyat_unique = p_fiyat_unique.groupby(["Date", "En Güncel Kod"]).agg({"Fiyat": "mean", "fiyat_gecisi": "max"}).reset_index()

---
# 0'ları doldururken en son yıl ve ayın ötesi de 0 ile dolduruyor. (Örn: Sales datasında 2021'in 3. ayına kadar data olduğu durumda 0'lar ile doldururken 2021 12. aya kadar 0 atıyor.
# Bu durumun önüne geçmek için aşağıdaki işlemler yapılmaktadır.
---

In [ ]:
backup_pas = pasifik_df_all2.copy()
backup_btt = btt_df_all2.copy()
backup_hor = horizon_saha_df_all2.copy()

In [ ]:
pas_max_year = pasifik_df_all["Yıl"].max()
pas_max_month = pasifik_df_all[pasifik_df_all["Yıl"] == pasifik_df_all["Yıl"].max()]["Ay"].max()

hor_max_year = horizon_saha_df_all["Yıl"].max()
hor_max_month = horizon_saha_df_all[horizon_saha_df_all["Yıl"] == horizon_saha_df_all["Yıl"].max()]["Ay"].max()

btt_max_year = btt_df_all["Yıl"].max()
btt_max_month = btt_df_all[btt_df_all["Yıl"] == btt_df_all["Yıl"].max()]["Ay"].max()

In [ ]:
pasifik_df_all2 = pasifik_df_all2[~((pasifik_df_all2["Yıl"] == pas_max_year) & (pasifik_df_all2["Ay"] > pas_max_month))].reset_index(drop=True)

In [ ]:
horizon_saha_df_all2 = horizon_saha_df_all2[~((horizon_saha_df_all2["Yıl"] == hor_max_year) & 
                                            (horizon_saha_df_all2["Ay"] > hor_max_month))].reset_index(drop=True)

In [ ]:
btt_df_all2 = btt_df_all2[~((btt_df_all2["Yıl"] == btt_max_year) & (btt_df_all2["Ay"] > btt_max_month))].reset_index(drop=True)

---

---

# Portföy

---

---

## Pasifik Portföy

In [ ]:
pasifik_portfoy_df2 = pd.merge(pasifik_portfoy_df, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left", left_on="Kod", right_on="Ürün Kodu")
pasifik_portfoy_df2["En Güncel Kod"] = pasifik_portfoy_df2["En Güncel Kod"].fillna(pasifik_portfoy_df2["Kod"])
pasifik_portfoy_df2.drop("Ürün Kodu", axis=1, inplace=True)
pasifik_portfoy_df2 = pasifik_portfoy_df2[pasifik_portfoy_df2["En Güncel Kod"] != "delist"].reset_index(drop=True)
pasifik_portfoy_df2["Portfoy"] = 1

## Horizon Portföy

In [ ]:
horizon_portfoy_df2 = pd.merge(horizon_portfoy_df, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left", left_on="Kod", right_on="Ürün Kodu")
horizon_portfoy_df2["En Güncel Kod"] = horizon_portfoy_df2["En Güncel Kod"].fillna(horizon_portfoy_df2["Kod"])
horizon_portfoy_df2.drop("Ürün Kodu", axis=1, inplace=True)
horizon_portfoy_df2 = horizon_portfoy_df2[horizon_portfoy_df2["En Güncel Kod"] != "delist"].reset_index(drop=True)
horizon_portfoy_df2["Portfoy"] = 1

In [ ]:
horizon_portfoy_df2 = horizon_portfoy_df2[~((horizon_portfoy_df2["Kod"] == 135901))]

## BTT Portföy

In [ ]:
btt_portfoy_df2 = pd.merge(btt_portfoy_df, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left", left_on="Kod", right_on="Ürün Kodu")
btt_portfoy_df2["En Güncel Kod"] = btt_portfoy_df2["En Güncel Kod"].fillna(btt_portfoy_df2["Kod"])
btt_portfoy_df2.drop("Ürün Kodu", axis=1, inplace=True)
btt_portfoy_df2 = btt_portfoy_df2[btt_portfoy_df2["En Güncel Kod"] != "delist"].reset_index(drop=True)
btt_portfoy_df2["Portfoy"] = 1

In [ ]:
btt_portfoy_df2 = btt_portfoy_df2[~((btt_portfoy_df2["Kod"] == 135901))]

---

---

# Portföy Kapsamındaki Sales Dataları

---

---

In [ ]:
pasifik_df_all3 = pd.merge(pasifik_df_all2,pasifik_portfoy_df2[["En Güncel Kod", "Portfoy"]], on="En Güncel Kod", how="left")
btt_df_all3 = pd.merge(btt_df_all2,btt_portfoy_df2[["En Güncel Kod", "Portfoy"]], on="En Güncel Kod", how="left")
horizon_saha_df_all3 = pd.merge(horizon_saha_df_all2,horizon_portfoy_df2[["En Güncel Kod", "Portfoy"]], on="En Güncel Kod", how="left")

In [ ]:
pasifik_df_all3["Portfoy"].fillna(0, inplace=True)
btt_df_all3["Portfoy"].fillna(0, inplace=True)
horizon_saha_df_all3["Portfoy"].fillna(0, inplace=True)

---
# Datalara Calender Eklenmesi
---

In [ ]:
calender_df.pop("DATE")

In [ ]:
pasifik_df_all3 = pd.merge(pasifik_df_all3, calender_df, on=["Yıl", "Ay"], how="left")
btt_df_all3 = pd.merge(btt_df_all3, calender_df, on=["Yıl", "Ay"], how="left")
horizon_saha_df_all3 = pd.merge(horizon_saha_df_all3, calender_df, on=["Yıl", "Ay"], how="left")

---
# Dataların Fiyat Ve Aktiviteler İle Birleştirilmesi
---

# Not:
---
### 1) BTT aktivite verisi için Horizon kısmındaki "Geleneksel Kanal" kullanılması istendi.
### 2) BTT fiyat geçişleri için Horizon fiyat geçişleri baz alındı.

In [ ]:
pasifik_df_all3 = pasifik_df_all3.merge(p_fiyat_unique, how="left", on=["Date", "En Güncel Kod"])

In [ ]:
pasifik_aktivite_df3.rename(columns={"Müşteri Grup": "Grup adı"}, inplace=True)
pasifik_df_all3 = pd.merge(pasifik_df_all3, pasifik_aktivite_df3[["Yıl", "Ay", "Grup adı", "En Güncel Kod", 
                                                                  "Raf Tavsiye Satış Fiyatı", "İndirimli Raf Satış Fiyatı", "İndirim %",
                                                                  "Aktivite Tipi"]], 
                           left_on=["Yıl", "Ay", "Grup adı", "En Güncel Kod"], 
                           right_on=["Yıl", "Ay", "Grup adı", "En Güncel Kod"], how="left")

In [ ]:
horizon_saha_df_all3 = horizon_saha_df_all3.merge(h_fiyat_unique, how="left", on=["Date", "En Güncel Kod"])

In [ ]:
horizon_saha_df_all3 = horizon_saha_df_all3.merge(saha_aktivite_detay3[['Ciro (Kull. İade Düş.)', 'Promosyon Tutarı', 
                                                                        'İskonto %', 'En Güncel Kod', "Yıl", "Ay", "Grup adı"]],
                                                  on=["En Güncel Kod", "Yıl", "Ay", "Grup adı"], how="left")

In [ ]:
btt_df_all3 = btt_df_all3.merge(h_fiyat_unique, how="left", on=["Date", "En Güncel Kod"])

In [ ]:
btt_aktivite = saha_aktivite_detay3[saha_aktivite_detay3["Grup adı"] == "GELENEKSEL KANAL"].reset_index(drop=True)

btt_df_all3 = btt_df_all3.merge(btt_aktivite[['Ciro (Kull. İade Düş.)', 'Promosyon Tutarı', 
                                              'İskonto %', 'En Güncel Kod', "Yıl", "Ay"]],
                                on=["En Güncel Kod", "Yıl", "Ay"], how="left")

---
# SKU Bazında İncelemeler
---

In [ ]:
sku = pd.concat([pasifik_df_all3[["Grup adı", "En Güncel Kod", "Portfoy"]], 
                 btt_df_all3[["Grup adı", "En Güncel Kod", "Portfoy"]], 
                 horizon_saha_df_all3[["Grup adı", "En Güncel Kod", "Portfoy"]]], axis=0, ignore_index=True)


only_sku = pd.concat([pasifik_df_all3["En Güncel Kod"], btt_df_all3["En Güncel Kod"], horizon_saha_df_all3["En Güncel Kod"]], 
                    axis=0, ignore_index=True)


kategori_all = pd.concat([pasifik_df_all3["Kategori Adı"], 
                 btt_df_all3["Kategori Adı"],
                 horizon_saha_df_all3["Kategori Adı"]], axis=0, ignore_index=True)


kategori_kanal = pd.concat([pasifik_df_all3[["Grup adı", "Kategori Adı"]], 
                 btt_df_all3[["Grup adı", "Kategori Adı"]],
                 horizon_saha_df_all3[["Grup adı", "Kategori Adı"]]], axis=0, ignore_index=True).drop_duplicates(subset=["Grup adı", "Kategori Adı"]).reset_index(drop=True)


only_sku = only_sku.drop_duplicates().sort_values().reset_index(drop=True)
sku = sku.drop_duplicates(subset=sku.columns.to_list()).sort_values(by=sku.columns.to_list()).reset_index(drop=True)
kategori_all = kategori_all.drop_duplicates().sort_values().reset_index(drop=True)
kategori_kanal = kategori_kanal.sort_values(by=kategori_kanal.columns.to_list()).reset_index(drop=True)

kategori_bazli_urun_sayilari = pd.concat([pasifik_df_all3[["Grup adı", "Kategori Adı", "En Güncel Kod"]], 
                 btt_df_all3[["Grup adı", "Kategori Adı", "En Güncel Kod"]],
                 horizon_saha_df_all3[["Grup adı", "Kategori Adı", "En Güncel Kod"]]], 
                           axis=0, ignore_index=True).drop_duplicates(subset=["Grup adı", 
                                                                              "Kategori Adı", 
                                                                              "En Güncel Kod"]).sort_values(by=["Grup adı", 
                                                                                                                "Kategori Adı", 
                                                                                                                "En Güncel Kod"]).reset_index(drop=True)

kategori_bazli_urun_sayilari = kategori_bazli_urun_sayilari.groupby(["Grup adı", "Kategori Adı"]).agg("count").reset_index()


writer_sku = pd.ExcelWriter("../reports/SKU_kodlari.xlsx", engine = 'xlsxwriter')

only_sku.to_excel(writer_sku, sheet_name='tum_sku', index=False)
sku.to_excel(writer_sku, sheet_name='grup_adina_gore_sku', index=False)
kategori_all.to_excel(writer_sku, sheet_name='kategori_tumu', index=False)
kategori_kanal.to_excel(writer_sku, sheet_name='kategori_kanallara_gore', index=False)
kategori_bazli_urun_sayilari.to_excel(writer_sku, sheet_name='kanal_kat_gore_sku_sayilari', index=False)


writer_sku.save()
writer_sku.close()

---

---

# Eşlenik Kod (En Güncel Kod) kısmında yer almayan Ürün Kod'ları

---

---

In [ ]:
diff_pasifik = list(set(eslenik_kod_df["En Güncel Kod"].unique()) - set(pasifik_df_all3["En Güncel Kod"].unique()))
diff_btt = list(set(eslenik_kod_df["En Güncel Kod"].unique()) - set(btt_df_all3["En Güncel Kod"].unique()))
diff_horizon = list(set(eslenik_kod_df["En Güncel Kod"].unique()) - set(horizon_saha_df_all3["En Güncel Kod"].unique()))

diff_pasifik = pd.DataFrame({"En Güncel Kod": diff_pasifik, "Kanal": ["Pasifik"]*len(diff_pasifik)})
diff_btt = pd.DataFrame({"En Güncel Kod": diff_btt, "Kanal": ["BTT"]*len(diff_btt)})
diff_horizon = pd.DataFrame({"En Güncel Kod": diff_horizon, "Kanal": ["Horizon"]*len(diff_horizon)})

pd.concat([diff_pasifik, diff_btt, diff_horizon], axis=0, ignore_index=True).to_excel("../reports/urun_kodu_historik_datada_olmayan_sku.xlsx", index=False)

---

---

# Fiyat Geçiş Bilgisi Verilmeyen SKU'lar

In [ ]:
p_fiyat_yok = pasifik_df_all3[pd.isnull(pasifik_df_all3["Fiyat"])][["En Güncel Kod", "Date"]].drop_duplicates().reset_index(drop=True)
p_fiyat_yok["Kanal"] = "Pasifik"
p_fiyat_yok.sort_values(by=["En Güncel Kod", "Date"], ignore_index=True, inplace=True)

In [ ]:
h_fiyat_yok = horizon_saha_df_all3[pd.isnull(horizon_saha_df_all3["Fiyat"])][["En Güncel Kod", "Date"]].drop_duplicates().reset_index(drop=True)
h_fiyat_yok["Kanal"] = "Horizon"
h_fiyat_yok.sort_values(by=["En Güncel Kod", "Date"], ignore_index=True, inplace=True)

In [ ]:
pd.concat([h_fiyat_yok, p_fiyat_yok], axis=0).to_excel("../reports/fiyat_bilgisi_verilmeyen_skular.xlsx", index=False)

# Sütun İsimlerini İngilizce Karaktere Çevirme

In [ ]:
chng_letters = list(zip(["ç", "ğ", "ı", "ö", "ş", "ü", " ", "%", ".", "(", ")", "-"], 
                        ["c", "g", "i", "o", "s", "u", "_", "", "", "", "", "_"]))

In [ ]:
pasifik_new_cols = []
for col in pasifik_df_all3.columns.str.lower():
    for letter in range(len(chng_letters)):
        col = col.replace(chng_letters[letter][0], chng_letters[letter][1])
        if letter == len(chng_letters) - 1:
            pasifik_new_cols.append(col)
        else:
            pass

In [ ]:
btt_new_cols = []
for col in btt_df_all3.columns.str.lower():
    for letter in range(len(chng_letters)):
        col = col.replace(chng_letters[letter][0], chng_letters[letter][1])
        if letter == len(chng_letters) - 1:
            btt_new_cols.append(col)
        else:
            pass

In [ ]:
horizon_new_cols = []
for col in horizon_saha_df_all3.columns.str.lower():
    for letter in range(len(chng_letters)):
        col = col.replace(chng_letters[letter][0], chng_letters[letter][1])
        if letter == len(chng_letters) - 1:
            horizon_new_cols.append(col)
        else:
            pass

In [ ]:
enflasyon_new_cols = []
for col in enflasyon_df.columns.str.lower():
    for letter in range(len(chng_letters)):
        col = col.replace(chng_letters[letter][0], chng_letters[letter][1])
        if letter == len(chng_letters) - 1:
            enflasyon_new_cols.append(col)
        else:
            pass

In [ ]:
pasifik_df_all3.columns = pasifik_new_cols

In [ ]:
btt_df_all3.columns = btt_new_cols

In [ ]:
horizon_saha_df_all3.columns = horizon_new_cols

In [ ]:
enflasyon_df.columns = enflasyon_new_cols

---
# Pasifikte Aktivite Tipi Verisi Eksik Olan Verilere "Yok" yazıldı
---

In [ ]:
pasifik_df_all3["aktivite_tipi"].fillna("Yok", inplace=True)

---
# Enflasyon Verilerinin Eklenmesi
---

In [ ]:
enflasyon_df = enflasyon_df[["yil", "ay", "enflasyon_etkisi"]].drop_duplicates(subset=["yil", "ay", "enflasyon_etkisi"], keep="first").reset_index(drop=True)

In [ ]:
pasifik_df_all3 = pasifik_df_all3.merge(enflasyon_df[["yil", "ay", "enflasyon_etkisi"]], on=["yil", "ay"], how="left")
horizon_saha_df_all3 = horizon_saha_df_all3.merge(enflasyon_df[["yil", "ay", "enflasyon_etkisi"]], on=["yil", "ay"], how="left")
btt_df_all3 = btt_df_all3.merge(enflasyon_df[["yil", "ay", "enflasyon_etkisi"]], on=["yil", "ay"], how="left")

---
# Gözlemlerin Sırasının Fiyatın Büyükten Küçüğe Sıralanmasına Göre Düzenlenmesi (SKU bazında fiyat benzerliği pattern'i yakalamak amacıyla yapıldı)
---

In [ ]:
sorter = pasifik_df_all3.groupby(["en_guncel_kod"]).agg({"fiyat": "max"}).reset_index().sort_values(by="fiyat", ascending=False)["en_guncel_kod"].to_list()
sorterIndex = dict(zip(sorter, range(len(sorter))))
pasifik_df_sorted = pasifik_df_all3.copy()
pasifik_df_sorted["sorted"] = pasifik_df_sorted["en_guncel_kod"].map(sorterIndex)
pasifik_df_sorted.sort_values(['sorted', "grup_adi", "yil", "ay"],
        ascending = [True]*4, inplace = True)
pasifik_df_sorted.drop('sorted', 1, inplace = True)
pasifik_df_sorted.reset_index(drop=True, inplace=True)

In [ ]:
sorter = horizon_saha_df_all3.groupby(["en_guncel_kod"]).agg({"fiyat": "max"}).reset_index().sort_values(by="fiyat", ascending=False)["en_guncel_kod"].to_list()
sorterIndex = dict(zip(sorter, range(len(sorter))))
horizon_saha_df_sorted = horizon_saha_df_all3.copy()
horizon_saha_df_sorted["sorted"] = horizon_saha_df_sorted["en_guncel_kod"].map(sorterIndex)
horizon_saha_df_sorted.sort_values(['sorted', "grup_adi", "yil", "ay"],
        ascending = [True]*4, inplace = True)
horizon_saha_df_sorted.drop('sorted', 1, inplace = True)
horizon_saha_df_sorted.reset_index(drop=True, inplace=True)

In [ ]:
sorter = btt_df_all3.groupby(["en_guncel_kod"]).agg({"fiyat": "max"}).reset_index().sort_values(by="fiyat", ascending=False)["en_guncel_kod"].to_list()
sorterIndex = dict(zip(sorter, range(len(sorter))))
btt_df_sorted = btt_df_all3.copy()
btt_df_sorted["sorted"] = btt_df_sorted["en_guncel_kod"].map(sorterIndex)
btt_df_sorted.sort_values(['sorted', "grup_adi", "yil", "ay"],
        ascending = [True]*4, inplace = True)
btt_df_sorted.drop('sorted', 1, inplace = True)
btt_df_sorted.reset_index(drop=True, inplace=True)

---
## Pasifik Aktivite: 0'dan küçük değerler alınmayacak
## Horziond Aktivite: %1 ve %35 arası değerler alınacak
---

In [ ]:
pasifik_df_sorted["indirim_"] = [0 if akt < 0 else akt for akt in pasifik_df_sorted["i̇ndirim_"]]
pasifik_df_sorted.drop("i̇ndirim_", axis=1, inplace=True)

In [ ]:
btt_df_sorted["iskonto_"] = [0 if ((akt >= 0.35) or (akt <=0.01)) else akt for akt in btt_df_sorted["i̇skonto_"]]
btt_df_sorted.drop("i̇skonto_", axis=1, inplace=True)

In [ ]:
horizon_saha_df_sorted["iskonto_"] = [0 if ((akt >= 0.35) or (akt <=0.01)) else akt for akt in horizon_saha_df_sorted["i̇skonto_"]]
horizon_saha_df_sorted.drop("i̇skonto_", axis=1, inplace=True)

---
## Satış Olmayan Aylar Flaglendi
---

In [ ]:
pasifik_df_sorted["satis_var"] = [0 if adet <= 1 else 1 for adet in pasifik_df_sorted["adet"]]

In [ ]:
btt_df_sorted["satis_var"] = [0 if adet <= 1 else 1 for adet in btt_df_sorted["adet"]]

In [ ]:
horizon_saha_df_sorted["satis_var"] = [0 if adet <= 1 else 1 for adet in horizon_saha_df_sorted["adet"]]

In [ ]:
df_pasifik = pasifik_df_sorted.copy()
df_btt = btt_df_sorted.copy()
df_horizon = horizon_saha_df_sorted.copy()

---
# SKU Sayilari
---

In [ ]:
sku = []
grup_adi = []
portfoy = []
oran = []
gozlem_sayisi = []
toplam_satir = []
baslangic_tarih = []
bitis_tarih = []
son_kac_ay_eksik = []
kanal = []
for idx, temp_df in df_pasifik[["grup_adi", "en_guncel_kod", "portfoy", "adet", "date"]].groupby(["grup_adi", "en_guncel_kod", "portfoy"]):
    sku.append(temp_df["en_guncel_kod"].iloc[0])
    grup_adi.append(temp_df["grup_adi"].iloc[0])
    portfoy.append(temp_df["portfoy"].iloc[0])
    oran.append(len(temp_df[temp_df["adet"] != 1]) / len(temp_df))
    gozlem_sayisi.append(len(temp_df[temp_df["adet"] != 1]))
    baslangic_tarih.append(temp_df[temp_df["adet"] != 1]["date"].min())
    bitis_tarih.append(temp_df[temp_df["adet"] != 1]["date"].max())
    toplam_satir.append(len(temp_df))
    son_kac_ay_eksik.append(round((datetime(2021, 3, 1) - temp_df[temp_df["adet"] != 1].date.max())/np.timedelta64(1, 'M'),1))
    kanal.append("Pasifik")

for idx, temp_df in df_horizon[["grup_adi", "en_guncel_kod", "portfoy", "adet", "date"]].groupby(["grup_adi", "en_guncel_kod", "portfoy"]):
    sku.append(temp_df["en_guncel_kod"].iloc[0])
    grup_adi.append(temp_df["grup_adi"].iloc[0])
    portfoy.append(temp_df["portfoy"].iloc[0])
    oran.append(len(temp_df[temp_df["adet"] != 1]) / len(temp_df))
    gozlem_sayisi.append(len(temp_df[temp_df["adet"] != 1]))
    baslangic_tarih.append(temp_df[temp_df["adet"] != 1]["date"].min())
    bitis_tarih.append(temp_df[temp_df["adet"] != 1]["date"].max())
    toplam_satir.append(len(temp_df))
    son_kac_ay_eksik.append(round((datetime(2021, 3, 1) - temp_df[temp_df["adet"] != 1].date.max())/np.timedelta64(1, 'M'),1))
    kanal.append("Horizon")

for idx, temp_df in df_btt[["grup_adi", "en_guncel_kod", "portfoy", "adet", "date"]].groupby(["grup_adi", "en_guncel_kod", "portfoy"]):
    sku.append(temp_df["en_guncel_kod"].iloc[0])
    grup_adi.append(temp_df["grup_adi"].iloc[0])
    portfoy.append(temp_df["portfoy"].iloc[0])
    oran.append(len(temp_df[temp_df["adet"] != 1]) / len(temp_df))
    gozlem_sayisi.append(len(temp_df[temp_df["adet"] != 1]))
    baslangic_tarih.append(temp_df[temp_df["adet"] != 1]["date"].min())
    bitis_tarih.append(temp_df[temp_df["adet"] != 1]["date"].max())
    toplam_satir.append(len(temp_df))
    son_kac_ay_eksik.append(round((datetime(2021, 3, 1) - temp_df[temp_df["adet"] != 1].date.max())/np.timedelta64(1, 'M'),1))
    kanal.append("BTT")

In [ ]:
sku_sayilari = pd.DataFrame({"sku": sku,
                             "grup_adi": grup_adi,
                             "portfoy": portfoy,
                             "gozlem_sayisi": gozlem_sayisi,
                             "toplam_satir": toplam_satir,
                             "oran": oran,
                             "baslangic_tarih": baslangic_tarih,
                             "bitis_tarih": bitis_tarih,
                             "son_kac_ay_eksik": son_kac_ay_eksik,
                             "kanal": kanal})

In [ ]:
sku_sayilari.sort_values(by=["sku", "grup_adi", "oran"], ignore_index=True, inplace=True)

In [ ]:
sku_sayilari.to_excel("../reports/sku_bazinda_sayilar.xlsx", index=False)

---
# Sütun isim uzunluğunun 32'yi geçmemesi için
---

In [ ]:
df_pasifik.columns = [i[:32] if len(i) > 32 else i for i in df_pasifik.columns]

In [ ]:
df_btt.columns = [i[:32] if len(i) > 32 else i for i in df_btt.columns]

In [ ]:
df_horizon.columns = [i[:32] if len(i) > 32 else i for i in df_horizon.columns]

---
# Sales datasındaki son yıl ve aya kadar ulaşmayan, yarıda kesilen verilerin, son tarihe kadar NaN ile doldurulması
## Sales verisindeki en son tarih Haziran 2021 ise, herhangi bir SKU'nun son gözlemi Şubat 2019'da olsa dahi Haizran 2021'e kadar devam ettiriliyor
---

In [ ]:
btt_and_horizon_cols_to_drop = ["grup_adi", "ana_kategori_adi", "kategori_adi", "marka_adi",
                                "urun_adi", "en_guncel_kod", "portfoy", "koli_i̇ci_adet", "koli", "kg", "tl", "adet", 
                                "fiyat", "fiyat_gecisi", "ciro_kull_i̇ade_dus", "promosyon_tutari",
                                "iskonto_", "satis_var"]

pasifik_cols_to_drop = ["grup_adi", "ana_kategori_adi", "kategori_adi", "marka_adi",
                        "urun_adi", "en_guncel_kod", "portfoy", "koli_i̇ci_adet", "koli", "kg", "tl", "adet",
                        "fiyat", "fiyat_gecisi", "raf_tavsiye_satis_fiyati", "i̇ndirimli_raf_satis_fiyati",
                        "aktivite_tipi", "indirim_", "satis_var"]

In [ ]:
pasifik_template = df_pasifik.drop(columns=pasifik_cols_to_drop, axis=1)
pasifik_template = pasifik_template.drop_duplicates(subset=pasifik_template.columns.to_list()).sort_values(by=["yil", "ay"])

In [ ]:
btt_template = df_btt.drop(columns=btt_and_horizon_cols_to_drop, axis=1)
btt_template = btt_template.drop_duplicates(subset=btt_template.columns.to_list()).sort_values(by=["yil", "ay"])

In [ ]:
horizon_template = df_horizon.drop(columns=btt_and_horizon_cols_to_drop, axis=1)
horizon_template = horizon_template.drop_duplicates(subset=horizon_template.columns.to_list()).sort_values(by=["yil", "ay"])

In [ ]:
pasifik_sku_fill = []
pasifik_grup_fill = []
for sku in df_pasifik["en_guncel_kod"].unique():
    for grup in df_pasifik["grup_adi"].unique():
        temp = df_pasifik[(df_pasifik["en_guncel_kod"] == sku) & (df_pasifik["grup_adi"] == grup)].reset_index(drop=True)
        if temp["date"].max() < pasifik_template["date"].max():
            pasifik_sku_fill.append(sku)
            pasifik_grup_fill.append(grup)
        else:
            pass

In [ ]:
btt_sku_fill = []
btt_grup_fill = []
for sku in df_btt["en_guncel_kod"].unique():
    for grup in df_btt["grup_adi"].unique():
        temp = df_btt[(df_btt["en_guncel_kod"] == sku) & (df_btt["grup_adi"] == grup)].reset_index(drop=True)
        if temp["date"].max() < btt_template["date"].max():
            btt_sku_fill.append(sku)
            btt_grup_fill.append(grup)
        else:
            pass

In [ ]:
horizon_sku_fill = []
horizon_grup_fill = []
for sku in df_horizon["en_guncel_kod"].unique():
    for grup in df_horizon["grup_adi"].unique():
        temp = df_horizon[(df_horizon["en_guncel_kod"] == sku) & (df_horizon["grup_adi"] == grup)].reset_index(drop=True)
        if temp["date"].max() < horizon_template["date"].max():
            horizon_sku_fill.append(sku)
            horizon_grup_fill.append(grup)
        else:
            pass

In [ ]:
dfs_pasifik_to_append = []

ffill_cols = ['grup_adi', 'ana_kategori_adi', 'kategori_adi',
             'marka_adi', 'urun_adi', 'en_guncel_kod', "portfoy"]

zero_fill_cols = ['koli_i̇ci_adet', 'koli', 'kg', 'tl', 'satis_var']

one_fill_cols = ["adet"]

for row in range(len(pasifik_sku_fill)):
    temp = df_pasifik[(df_pasifik["en_guncel_kod"] == pasifik_sku_fill[row]) & 
                      (df_pasifik["grup_adi"] == pasifik_grup_fill[row])].reset_index(drop=True)
    
    pasifik_template2 = pasifik_template[pasifik_template["date"] > temp.date.max()].reset_index(drop=True)
    
    df_to_append = temp.merge(pasifik_template2, how="outer", on=list(set(temp).intersection(set(pasifik_template2))))
    for f in ffill_cols:
        df_to_append[f].fillna(method="ffill", inplace=True)
    
    for z in zero_fill_cols:
        df_to_append[z].fillna(0, inplace=True)
    
    for o in one_fill_cols:
        df_to_append[o].fillna(1, inplace=True)
    dfs_pasifik_to_append.append(df_to_append)

In [ ]:
dfs_horizon_to_append = []

ffill_cols = ['grup_adi', 'ana_kategori_adi', 'kategori_adi',
             'marka_adi', 'urun_adi', 'en_guncel_kod', "portfoy"]

zero_fill_cols = ['koli_i̇ci_adet', 'koli', 'kg', 'tl', 'satis_var']

one_fill_cols = ["adet"]

for row in range(len(horizon_sku_fill)):
    temp = df_horizon[(df_horizon["en_guncel_kod"] == horizon_sku_fill[row]) & 
                      (df_horizon["grup_adi"] == horizon_grup_fill[row])].reset_index(drop=True)
    
    horizon_template2 = horizon_template[horizon_template["date"] > temp.date.max()].reset_index(drop=True)
    
    df_to_append = temp.merge(horizon_template2, how="outer", on=list(set(temp).intersection(set(horizon_template2))))
    for f in ffill_cols:
        df_to_append[f].fillna(method="ffill", inplace=True)
    
    for z in zero_fill_cols:
        df_to_append[z].fillna(0, inplace=True)
    
    for o in one_fill_cols:
        df_to_append[o].fillna(1, inplace=True)
    dfs_horizon_to_append.append(df_to_append)

In [ ]:
dfs_btt_to_append = []

ffill_cols = ['grup_adi', 'ana_kategori_adi', 'kategori_adi',
             'marka_adi', 'urun_adi', 'en_guncel_kod', "portfoy"]

zero_fill_cols = ['koli_i̇ci_adet', 'koli', 'kg', 'tl', 'satis_var']

one_fill_cols = ["adet"]

for row in range(len(btt_sku_fill)):
    temp = df_btt[(df_btt["en_guncel_kod"] == btt_sku_fill[row]) & 
                  (df_btt["grup_adi"] == btt_grup_fill[row])].reset_index(drop=True)
    
    btt_template2 = btt_template[btt_template["date"] > temp.date.max()].reset_index(drop=True)
    
    df_to_append = temp.merge(btt_template2, how="outer", on=list(set(temp).intersection(set(btt_template2))))
    for f in ffill_cols:
        df_to_append[f].fillna(method="ffill", inplace=True)
    
    for z in zero_fill_cols:
        df_to_append[z].fillna(0, inplace=True)
    
    for o in one_fill_cols:
        df_to_append[o].fillna(1, inplace=True)
    dfs_btt_to_append.append(df_to_append)

In [ ]:
df_pasifik2 = df_pasifik.copy()
df_horizon2 = df_horizon.copy()
df_btt2 = df_btt.copy()

In [ ]:
dfs_pasifik_to_append = pd.concat(dfs_pasifik_to_append)
dfs_horizon_to_append = pd.concat(dfs_horizon_to_append)
dfs_btt_to_append = pd.concat(dfs_btt_to_append)

In [ ]:
df_pasifik3 = pd.concat([df_pasifik2, dfs_pasifik_to_append], axis=0, ignore_index=True)
df_pasifik3.drop_duplicates(subset=df_pasifik3.columns.to_list(), ignore_index=True, inplace=True)

df_horizon3 = pd.concat([df_horizon2, dfs_horizon_to_append], axis=0, ignore_index=True)
df_horizon3.drop_duplicates(subset=df_horizon3.columns.to_list(), ignore_index=True, inplace=True)

df_btt3 = pd.concat([df_btt2, dfs_btt_to_append], axis=0, ignore_index=True)
df_btt3.drop_duplicates(subset=df_btt3.columns.to_list(), ignore_index=True, inplace=True)

In [ ]:
p_fiyat_unique.rename(columns={"Date": "date", "En Güncel Kod": "en_guncel_kod", "Fiyat": "fiyat"}, inplace=True)

In [ ]:
h_fiyat_unique.rename(columns={"Date": "date", "En Güncel Kod": "en_guncel_kod", "Fiyat": "fiyat"}, inplace=True)

In [ ]:
df_pasifik4 = df_pasifik3.drop(columns=['fiyat', 'fiyat_gecisi'], axis=1)
df_pasifik4 = df_pasifik4.merge(p_fiyat_unique, how="left", on=["date", "en_guncel_kod"])

df_horizon4 = df_horizon3.drop(columns=['fiyat', 'fiyat_gecisi'], axis=1)
df_horizon4 = df_horizon4.merge(h_fiyat_unique, how="left", on=["date", "en_guncel_kod"])

df_btt4 = df_btt3.drop(columns=['fiyat', 'fiyat_gecisi'], axis=1)
df_btt4 = df_btt4.merge(h_fiyat_unique, how="left", on=["date", "en_guncel_kod"])

In [ ]:
chng_letters = list(zip(["ç", "ğ", "ı", "ö", "ş", "ü", " ", "%", ".", "(", ")", "-"], 
                        ["c", "g", "i", "o", "s", "u", "_", "", "", "", "", "_"]))

In [ ]:
p_aktivite_new_cols = []
for col in pasifik_aktivite_df3.columns.str.lower():
    for letter in range(len(chng_letters)):
        col = col.replace(chng_letters[letter][0], chng_letters[letter][1])
        if letter == len(chng_letters) - 1:
            p_aktivite_new_cols.append(col)
        else:
            pass

In [ ]:
h_aktivite_new_cols = []
for col in saha_aktivite_detay3.columns.str.lower():
    for letter in range(len(chng_letters)):
        col = col.replace(chng_letters[letter][0], chng_letters[letter][1])
        if letter == len(chng_letters) - 1:
            h_aktivite_new_cols.append(col)
        else:
            pass

In [ ]:
pasifik_aktivite_df3.columns = p_aktivite_new_cols

In [ ]:
saha_aktivite_detay3.columns = h_aktivite_new_cols

In [ ]:
pasifik_aktivite_df3.rename(columns={"i̇ndirim_": "indirim_"}, inplace=True)

In [ ]:
saha_aktivite_detay3.rename(columns={"i̇i̇skonto_": "iskonto_"}, inplace=True)

In [ ]:
df_pasifik4 = df_pasifik4.drop(columns=['raf_tavsiye_satis_fiyati', 'i̇ndirimli_raf_satis_fiyati', 
                                        'indirim_', 'aktivite_tipi'], axis=1)

In [ ]:
df_horizon4 = df_horizon4.drop(columns=['ciro_kull_i̇ade_dus', 'promosyon_tutari', 'iskonto_'], axis=1)

In [ ]:
df_btt4 = df_btt4.drop(columns=['ciro_kull_i̇ade_dus', 'promosyon_tutari', 'iskonto_'], axis=1)

In [ ]:
pasifik_aktivite_df3.rename(columns={"Müşteri Grup": "Grup adı"}, inplace=True)
df_pasifik5 = pd.merge(df_pasifik4, pasifik_aktivite_df3[['yil', 'ay', 'grup_adi', 'en_guncel_kod', 
                                                          'raf_tavsiye_satis_fiyati', 'i̇ndirimli_raf_satis_fiyati', 'indirim_', 
                                                          'aktivite_tipi']], 
                           left_on=['yil', 'ay', 'grup_adi', 'en_guncel_kod'], 
                           right_on=['yil', 'ay', 'grup_adi', 'en_guncel_kod'], how="left")

In [ ]:
df_horizon5 = df_horizon4.merge(saha_aktivite_detay3[['yil', 'ay', 'grup_adi', 'ciro_kull_i̇ade_dus', 
                                                      'promosyon_tutari', 'i̇skonto_', 'en_guncel_kod']],
                                on=['en_guncel_kod', 'yil', 'ay', 'grup_adi'], how="left")

In [ ]:
btt_aktivite = saha_aktivite_detay3[saha_aktivite_detay3["grup_adi"] == "GELENEKSEL KANAL"].reset_index(drop=True)
btt_aktivite["grup_adi"] = "BTT"

df_btt5 = df_btt4.merge(btt_aktivite[['yil', 'ay', 'grup_adi', 'ciro_kull_i̇ade_dus', 
                                      'promosyon_tutari', 'i̇skonto_', 'en_guncel_kod']],
                        on=['en_guncel_kod', 'yil', 'ay', 'grup_adi'], how="left")

---
## Pasifik Aktivite: 0'dan küçük değerler alınmayacak
## Horziond Aktivite: %1 ve %35 arası değerler alınacak
---

In [ ]:
df_pasifik5["indirim__"] = [0 if akt < 0 else akt for akt in df_pasifik5["indirim_"]]
df_pasifik5.drop("indirim_", axis=1, inplace=True)

In [ ]:
df_btt5["iskonto__"] = [0 if ((akt >= 0.35) or (akt <=0.01)) else akt for akt in df_btt5["i̇skonto_"]]
df_btt5.drop("i̇skonto_", axis=1, inplace=True)

In [ ]:
df_horizon5["iskonto__"] = [0 if ((akt >= 0.35) or (akt <=0.01)) else akt for akt in df_horizon5["i̇skonto_"]]
df_horizon5.drop("i̇skonto_", axis=1, inplace=True)

---
# Gözlemlerin Sırasının Fiyatın Büyükten Küçüğe Sıralanmasına Göre Düzenlenmesi (SKU bazında fiyat benzerliği pattern'i yakalamak amacıyla yapıldı)
---

In [ ]:
sorter = df_pasifik5.groupby(["en_guncel_kod"]).agg({"fiyat": "max"}).reset_index().sort_values(by="fiyat", ascending=False)["en_guncel_kod"].to_list()
sorterIndex = dict(zip(sorter, range(len(sorter))))
pasifik_df_sorted = df_pasifik5.copy()
pasifik_df_sorted["sorted"] = pasifik_df_sorted["en_guncel_kod"].map(sorterIndex)
pasifik_df_sorted.sort_values(['sorted', "grup_adi", "yil", "ay"],
                              ascending = [True]*4, inplace = True)
pasifik_df_sorted.drop('sorted', 1, inplace = True)
pasifik_df_sorted.reset_index(drop=True, inplace=True)

In [ ]:
sorter = df_horizon5.groupby(["en_guncel_kod"]).agg({"fiyat": "max"}).reset_index().sort_values(by="fiyat", ascending=False)["en_guncel_kod"].to_list()
sorterIndex = dict(zip(sorter, range(len(sorter))))
horizon_saha_df_sorted = df_horizon5.copy()
horizon_saha_df_sorted["sorted"] = horizon_saha_df_sorted["en_guncel_kod"].map(sorterIndex)
horizon_saha_df_sorted.sort_values(['sorted', "grup_adi", "yil", "ay"],
                                   ascending = [True]*4, inplace = True)
horizon_saha_df_sorted.drop('sorted', 1, inplace = True)
horizon_saha_df_sorted.reset_index(drop=True, inplace=True)

In [ ]:
sorter = df_btt5.groupby(["en_guncel_kod"]).agg({"fiyat": "max"}).reset_index().sort_values(by="fiyat", ascending=False)["en_guncel_kod"].to_list()
sorterIndex = dict(zip(sorter, range(len(sorter))))
btt_df_sorted = df_btt5.copy()
btt_df_sorted["sorted"] = btt_df_sorted["en_guncel_kod"].map(sorterIndex)
btt_df_sorted.sort_values(['sorted', "grup_adi", "yil", "ay"],
        ascending = [True]*4, inplace = True)
btt_df_sorted.drop('sorted', 1, inplace = True)
btt_df_sorted.reset_index(drop=True, inplace=True)

---
# Yarışma Verisinin Eklenmesi
---

In [ ]:
yarisma_df

In [ ]:
enflasyon_df = pd.read_excel("enflasyon.xlsx")

In [ ]:
enflasyon_df

In [ ]:
yarisma_df = pd.read_excel("../data/2018-2021 Yarışmaları_v2.xlsx", sheet_name="yarisma_historik_data")

---
# Datanın dışarıya aktarılması
### Fiyata Göre Sıralanmış Datanın Dışarıya Aktarılması
---

In [158]:
pasifik_df_sorted.to_excel("../data/_pasifik_sales_portfoy.xlsx", index=False)

In [159]:
btt_df_sorted.to_excel("../data/_btt_sales_portfoy.xlsx", index=False)

In [160]:
horizon_saha_df_sorted.to_excel("../data/_horizon_sales_portfoy.xlsx", index=False)

In [161]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("\Data prep çalışma süresi: "+"{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

\Data prep çalışma süresi: 450920:13:38.20
